In [4]:
from backtesting import Backtest, Strategy
from vnstock import *
from backtesting.lib import crossover, TrailingStrategy
import pandas as pd
import numpy as np
from backtesting.test import SMA


c:\Users\maphq\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
c:\Users\maphq\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [6]:
data = stock_historical_data(symbol='HPG', 
                             start_date='2015-01-01', 
                             end_date='2022-12-31', 
                             resolution='1D', 
                             type='stock', 
                             beautify=True, 
                             decor=True, 
                             source='DNSE')
data

,Open,High,Low,Close,Volume,Ticker
Time,,,,,,
2015-01-05,4580,4670,4540,4580,207020,HPG
2015-01-06,4490,4620,4490,4620,260380,HPG
2015-01-07,4620,4670,4580,4580,232770,HPG
2015-01-08,4580,4620,4540,4580,107380,HPG
2015-01-09,4580,4670,4540,4670,409230,HPG
...,...,...,...,...,...,...
2022-12-27,17100,17450,17100,17400,8344700,HPG
2022-12-27,17400,18250,17350,18250,23655000,HPG
2022-12-28,18400,18500,17950,18200,14311200,HPG


In [18]:
class SmaCross(TrailingStrategy):
    def init(self):
        super().init()
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 50)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()
        self.set_tp_sl(tsl_th=0.05, tsl_stop=0.02, sl_stop=0.05, tp_stop=0.15)
    
    def set_tp_sl(self, tsl_th, tsl_stop, sl_stop, tp_stop):
        self.tsl_th = tsl_th
        self.tsl_stop = tsl_stop
        self.sl_stop = sl_stop
        self.tp_stop = tp_stop

    def log_trade(self, trade):

        pass

    def on_order_executed(self, order):
        self.log_trade(order)

    def on_position_opened(self, position):
        entry_price = position.entry_price
        if position.is_long:
            self.sell(sl=entry_price * (1 - self.sl_stop), tp=entry_price * (1 + self.tp_stop))
            if self.data.Close[-1] > entry_price * (1 + self.tsl_th):
                self.sell(sl=entry_price * (1 + self.tsl_stop))
        else:
            self.buy(sl=entry_price * (1 + self.sl_stop), tp=entry_price * (1 - self.tp_stop))
            if self.data.Close[-1] < entry_price * (1 - self.tsl_th):
                self.buy(sl=entry_price * (1 - self.tsl_stop))

In [19]:
bt = Backtest(data, SmaCross, commission=.002, exclusive_orders=True)

stats = bt.run()
bt.plot(plot_width=1000, plot_equity=True, plot_pl=True)

c:\Users\maphq\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
C:\Users\maphq\AppData\Local\Temp\ipykernel_24628\1861870915.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross, commission=.002, exclusive_orders=True)
C:\Users\maphq\AppData\Local\Temp\ipykernel_24628\1861870915.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, SmaCross, commission=.002, exclusive_orders=True)


TypeError: '>=' not supported between instances of 'float' and 'datetime.timedelta'